<a href="https://colab.research.google.com/github/minjaecd123/study/blob/master/20250331_1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 128mb 데이터

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/syon/Data/merge_trends_weather_long_dl.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473364 entries, 0 to 473363
Data columns (total 60 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           473364 non-null  object 
 1   GroupName      473364 non-null  object 
 2   Gender         473364 non-null  object 
 3   Age_Group      473364 non-null  object 
 4   Search_Count   473364 non-null  float64
 5   STN            473364 non-null  int64  
 6   WS_AVG         472716 non-null  float64
 7   WR_DAY         472572 non-null  float64
 8   WD_MAX         473004 non-null  float64
 9   WS_MAX         473004 non-null  float64
 10  WS_MAX_TM      473004 non-null  float64
 11  WD_INS         472932 non-null  float64
 12  WS_INS         472932 non-null  float64
 13  WS_INS_TM      472932 non-null  float64
 14  TA_AVG         473328 non-null  float64
 15  TA_MAX         473364 non-null  float64
 16  TA_MAX_TM      473328 non-null  float64
 17  TA_MIN         473004 non-nul

In [ ]:
# ✅ XGBoost 음식 추천 모델 전체 코드 (개선된 열 기준)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, PowerTransformer
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
import xgboost as xgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# === 1. 데이터 로딩 및 정리 ===
df = pd.read_csv('/content/drive/MyDrive/syon/Data/merge_trends_weather_long_dl.csv')
df.columns = df.columns.str.strip()

# === 2. 날짜 → 월 ===
df['Month'] = pd.to_datetime(df['Date']).dt.month

# === 3. 날씨 피처 매핑 ===
weather_columns = {
    'Temperature': 'TA_AVG',
    'Humidity': 'HM_AVG',
    'Precipitation': 'RN_DAY',
    'Snowfall': 'SD_MAX',
    'WindSpeed': 'WS_AVG',
    'DewPoint': 'TD_AVG',
    'Pressure': 'PA_AVG',
    'CloudAmount': 'CA_TOT',
    'Sunshine': 'SS_DUR'
}
actual_weather_columns = list(weather_columns.values())

# === 4. 누락된 컬럼 확인 ===
missing = [col for col in actual_weather_columns if col not in df.columns]
if missing:
    raise KeyError(f"\u274c 누락된 컬럼: {missing}")

# === 5. 이상치 및 결측치 처리 ===
df[actual_weather_columns] = df[actual_weather_columns].replace(-9.0, 0)
df[actual_weather_columns] = df[actual_weather_columns].fillna(0)

# === 6. 검색량 변환 ===
pt = PowerTransformer(method='yeo-johnson')
df['Search_Count'] = pt.fit_transform(df[['Search_Count']])

# === 7. 범주형 인코딩 ===
label_cols = ['GroupName', 'Gender', 'Age_Group']
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df['GroupName_Label'] = label_encoders['GroupName'].inverse_transform(df['GroupName'])

# === 8. 날씨 클러스터링 ===
scaler = StandardScaler()
weather_scaled = scaler.fit_transform(df[actual_weather_columns])
kmeans = KMeans(n_clusters=5, random_state=42)
df['Weather_Cluster'] = kmeans.fit_predict(weather_scaled)

weather_labels = {
    0: '맑음/보통', 1: '고기압/건조', 2: '흐림/습함', 3: '추움/강풍', 4: '더움/일조많음'
}
df['Weather_Cluster_Label'] = df['Weather_Cluster'].map(weather_labels)
df['Weather_Cluster'] = LabelEncoder().fit_transform(df['Weather_Cluster_Label'])

# === 9. 모델 피처 구성 ===
features = ['Search_Count', 'STN', 'Month', 'Gender', 'Age_Group', 'Weather_Cluster']
X = df[features]
y = df['GroupName']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === 10. XGBoost 모델 학습 ===
xgb_model = xgb.XGBClassifier(
    random_state=42, tree_method='hist', use_label_encoder=False, eval_metric='mlogloss'
)

param_dist = {
    'n_estimators': [100],
    'max_depth': [10, 20],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=4,
    scoring='f1_weighted',
    cv=2,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# === 11. 정확도 평가 ===
y_pred = best_model.predict(X_test)
print("\nBest Parameters:", random_search.best_params_)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_encoders['GroupName'].classes_))



Fitting 2 folds for each of 4 candidates, totalling 8 fits


ERROR:asyncio:Exception in callback BaseAsyncIOLoop._handle_events(29, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(29, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 202, in _handle_events
    handler_func(fileobj, events)
  File "/usr/local/lib/python3.11/dist-packages/zmq/eventloop/zmqstream.py", line 578, in _handle_events
    self._handle_recv()
  File "/usr/local/lib/python3.11/dist-packages/zmq/eventloop/zmqstream.py", line 607, in _handle_recv
    self._run_callback(callback, msg)
  File "/usr/local/lib/python3.11/dist-packages/zmq/eventloop/zmqstream.py", line 557, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/iostream.py", line 120, in _handle_event
    event_f()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/iostr

In [ ]:
# === 11. 정확도 평가 ===
y_pred = best_model.predict(X_test)
print("\nBest Parameters:", random_search.best_params_)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_encoders['GroupName'].classes_))


Best Parameters: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1}

Classification Report:

              precision    recall  f1-score   support

         면요리       0.59      0.56      0.58     15778
      밥/죽/덮밥       0.48      0.43      0.46     15779
       볶음/구이       0.48      0.57      0.52     15779
     브런치/샐러드       0.52      0.49      0.51     15779
      안주/보양식       0.93      0.92      0.92     15779
      찌개/국/탕       0.81      0.86      0.84     15779

    accuracy                           0.64     94673
   macro avg       0.64      0.64      0.64     94673
weighted avg       0.64      0.64      0.64     94673



# 관계없음 기상청 api

In [ ]:
import requests
from datetime import datetime, timedelta

def get_kma_weather(service_key, nx=60, ny=127):
    """
    기상청 초단기실황 API 호출 함수
    :param service_key: 인증키 (URL 인코딩된 상태여야 함)
    :param nx: 격자 X 좌표 (예: 서울 60)
    :param ny: 격자 Y 좌표 (예: 서울 127)
    :return: dict 형태의 날씨 실황 정보
    """
    # 기준 날짜/시간 계산 (현재 시각에서 40분 전 기준)
    now = datetime.now()
    base_time = (now - timedelta(minutes=40)).strftime('%H00')
    base_date = now.strftime('%Y%m%d')

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    params = {
        'serviceKey': service_key,
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'JSON',          # ✅ JSON 형식으로 받기
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        try:
            items = response.json()['response']['body']['items']['item']
            # 필요한 항목 추출
            weather = {item['category']: item['obsrValue'] for item in items}
            return weather
        except Exception as e:
            print("❌ 데이터 파싱 오류:", e)
            return None
    else:
        print(f"❌ 요청 실패: {response.status_code}")
        return None

# 예시 사용
service_key = '9/J+JAxSAQW37NA9zqY7YcAMkvZuUyZpDaGEfASZzlJFQQndFFdbWDaCZ4aiMWBp4NwNZOI2bMPk5H3NU0aq/g=='  # URL 인코딩된 키
weather = get_kma_weather(service_key)

if weather:
    print("✅ 현재 실황 정보:")
    for key, value in weather.items():
        print(f"{key}: {value}")


✅ 현재 실황 정보:
PTY: 0
REH: 69
RN1: 0
T1H: 3.1
UUU: -1.3
VEC: 47
VVV: -1.2
WSD: 1.9
